# Идея проекта, выборка и предобработка данных
<hr>

> TODO: ссылки на методологию анализа фейков по гистограммам, распознавание лиц в OpenCV, исходный датасет

# Обработка данных
<hr>

> TODO: объем и баланс, хар-ки данных на каждом этапе

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [30]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_json("../dfdc_dataframes/df_total_0_4_compact_frame_12.json")

In [33]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / x)

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 281)


,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
1021,225,srpvgysqdx.mp4,True,90000,7.155311,8.844331,10.716933,13.017684,15.009092,19.157002,...,1.851852,3.086420,11.419753,7.098765,0.272222,0.000011,0.013730,0.002376,5.000000,3.673469
773,312,jgbpmnqoxl.mp4,True,173056,25.678636,29.652559,20.355680,19.148881,18.744480,19.319099,...,1.562500,4.861111,9.027778,2.951389,0.121348,0.000006,0.020572,0.000945,12.017778,8.240762
1559,763,evzwnwrjgz.mp4,True,169744,6.938623,10.351923,11.676719,11.966312,10.388471,7.773653,...,5.208333,3.819444,1.562500,4.687500,0.052432,0.000006,0.047697,0.000727,54.756129,19.072360
170,180,ufaigyxttb.mp4,True,138384,3.993700,9.109385,8.136871,11.190025,11.678655,14.152406,...,1.446281,8.677686,6.198347,9.917355,0.101168,0.000007,0.040601,0.001017,18.451200,9.884571
471,486,ansbtntiqu.mp4,True,44100,12.247147,47.052405,37.522892,24.092318,11.428655,10.462047,...,0.694444,0.694444,1.388889,0.000000,0.464853,0.000023,0.023839,0.004961,3.500000,2.151220


In [34]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
True,100465.923232,45.100205,556.734376,0.179548,0.2885,0.000015,0.024697,0.003062,11.339692,5.732262


In [35]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
False,92126.343434,49.182392,534.771606,0.200743,0.308146,0.000016,0.022526,0.003057,8.383215,4.79785


In [36]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_8_hist_noise_3,12.842086,9.325458,0.158639,3.516628
index,541.650505,393.331313,0.158633,148.319192
pt_57_hist_noise_3,16.081387,11.789096,0.154008,4.292291
pt_7_hist_noise_3,13.834530,10.155915,0.153337,3.678615
overall_face_hist_noise_0.5_reversed,8.383215,11.339692,0.149901,2.956477
pt_6_hist_noise_3,12.170309,9.053778,0.146839,3.116531
pt_3_hist_noise_3,20.559039,15.441546,0.142150,5.117493
pt_58_hist_noise_3,17.242200,13.181953,0.133455,4.060246
pt_65_hist_noise_3,22.827112,17.483956,0.132548,5.343156
pt_39_hist_noise_3,23.890869,18.304631,0.132389,5.586239


In [37]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
face_size_px_reversed,0.000016,0.000015,0.000031
overall_face_hist_std_reversed,0.003057,0.003062,0.006119
overall_face_std_reversed,0.022526,0.024697,0.047223
overall_face_hist_noise_0.5,0.200743,0.179548,0.380291
overall_face_hist_noise_3,0.308146,0.288500,0.596646
pt_8_hist_noise_0.5,0.897199,1.025667,1.922866
pt_7_hist_noise_0.5,1.001284,1.112007,2.113291
pt_9_hist_noise_0.5,1.018589,1.108013,2.126602
pt_10_hist_noise_0.5,1.137544,1.232738,2.370281
pt_6_hist_noise_0.5,1.139390,1.304300,2.443690


In [38]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_8_hist_noise_3,3.516628,22.167544,0.158639
index,148.319192,934.981818,0.158633
pt_57_hist_noise_3,4.292291,27.870483,0.154008
pt_7_hist_noise_3,3.678615,23.990445,0.153337
overall_face_hist_noise_0.5_reversed,2.956477,19.722908,0.149901
pt_6_hist_noise_3,3.116531,21.224086,0.146839
pt_3_hist_noise_3,5.117493,36.000585,0.142150
pt_58_hist_noise_3,4.060246,30.424153,0.133455
pt_65_hist_noise_3,5.343156,40.311067,0.132548
pt_39_hist_noise_3,5.586239,42.195500,0.132389


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [39]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(7).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(7).index) +
    list(df_smart.filter(like='std', axis=0).head(5).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

26


['pt_36_std',
 'pt_3_std',
 'overall_face_std_reversed',
 'pt_58_hist_noise_3',
 'pt_39_std',
 'pt_23_hist_noise_0.5',
 'overall_face_hist_std',
 'overall_face_hist_noise_3_reversed',
 'pt_7_hist_noise_3',
 'pt_24_hist_noise_0.5',
 'pt_12_hist_noise_0.5',
 'pt_8_hist_noise_0.5',
 'pt_3_hist_noise_3',
 'pt_1_std',
 'overall_face_hist_noise_0.5_reversed',
 'pt_57_hist_noise_3',
 'pt_65_hist_noise_3',
 'pt_6_hist_noise_0.5',
 'face_size_px_reversed',
 'pt_6_hist_noise_3',
 'pt_21_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'overall_face_hist_std_reversed',
 'pt_8_hist_noise_3',
 'pt_41_std',
 'overall_face_hist_noise_0.5']

## Разбивка данных на train и test сплиты

In [40]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
1021,225,True,90000,7.155311,8.844331,10.716933,13.017684,15.009092,19.157002,13.268215,...,1.851852,3.086420,11.419753,7.098765,0.272222,0.000011,0.013730,0.002376,5.000000,3.673469
773,312,True,173056,25.678636,29.652559,20.355680,19.148881,18.744480,19.319099,19.724832,...,1.562500,4.861111,9.027778,2.951389,0.121348,0.000006,0.020572,0.000945,12.017778,8.240762
1559,763,True,169744,6.938623,10.351923,11.676719,11.966312,10.388471,7.773653,6.508004,...,5.208333,3.819444,1.562500,4.687500,0.052432,0.000006,0.047697,0.000727,54.756129,19.072360
170,180,True,138384,3.993700,9.109385,8.136871,11.190025,11.678655,14.152406,9.990499,...,1.446281,8.677686,6.198347,9.917355,0.101168,0.000007,0.040601,0.001017,18.451200,9.884571
471,486,True,44100,12.247147,47.052405,37.522892,24.092318,11.428655,10.462047,7.523545,...,0.694444,0.694444,1.388889,0.000000,0.464853,0.000023,0.023839,0.004961,3.500000,2.151220


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [42]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,pt_36_std,pt_3_std,overall_face_std_reversed,pt_58_hist_noise_3,pt_39_std,pt_23_hist_noise_0.5,overall_face_hist_std,overall_face_hist_noise_3_reversed,pt_7_hist_noise_3,pt_24_hist_noise_0.5,...,pt_65_hist_noise_3,pt_6_hist_noise_0.5,face_size_px_reversed,pt_6_hist_noise_3,pt_21_hist_noise_0.5,overall_face_hist_noise_3,overall_face_hist_std_reversed,pt_8_hist_noise_3,pt_41_std,overall_face_hist_noise_0.5
1021,26.519326,28.893737,0.013730,17.901235,17.157834,1.851852,420.870002,3.673469,4.629630,4.629630,...,13.888889,0.308642,0.000011,3.086420,2.469136,0.272222,0.002376,3.703704,17.329702,0.200000
773,53.338586,30.850342,0.020572,6.944444,31.676862,3.125000,1058.012520,8.240762,14.930556,3.645833,...,11.805556,6.423611,0.000006,12.847222,1.909722,0.121348,0.000945,14.930556,49.485074,0.083210
1559,7.948121,5.281147,0.047697,2.083333,13.167363,0.173611,1375.408194,19.072360,7.465278,0.347222,...,5.729167,1.041667,0.000006,6.423611,0.694444,0.052432,0.000727,4.340278,13.550020,0.018263
170,5.205979,4.894074,0.040601,2.892562,6.186786,0.826446,983.628703,9.884571,0.826446,1.033058,...,6.818182,0.413223,0.000007,0.826446,0.206612,0.101168,0.001017,4.958678,5.248916,0.054197
471,26.948078,7.138173,0.023839,25.000000,14.048747,0.000000,201.559932,2.151220,18.055556,0.000000,...,27.083333,0.000000,0.000023,13.194444,0.000000,0.464853,0.004961,5.555556,19.787411,0.285714


In [43]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [44]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [179]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=420)

In [180]:
pd.DataFrame(X_train).head()

,pt_36_std,pt_3_std,overall_face_std_reversed,pt_58_hist_noise_3,pt_39_std,pt_23_hist_noise_0.5,overall_face_hist_std,overall_face_hist_noise_3_reversed,pt_7_hist_noise_3,pt_24_hist_noise_0.5,...,pt_65_hist_noise_3,pt_6_hist_noise_0.5,face_size_px_reversed,pt_6_hist_noise_3,pt_21_hist_noise_0.5,overall_face_hist_noise_3,overall_face_hist_std_reversed,pt_8_hist_noise_3,pt_41_std,overall_face_hist_noise_0.5
1874,22.391359,42.322430,0.017269,5.769231,16.712470,3.106509,1101.052972,9.913474,3.698225,4.733728,...,16.863905,0.739645,0.000005,1.479290,2.810651,0.100873,0.000908,3.698225,25.749034,0.059993
2800,19.501571,26.143270,0.026368,13.842975,18.582243,4.752066,783.141879,8.060952,1.033058,0.000000,...,17.975207,0.413223,0.000007,2.066116,2.685950,0.124055,0.001277,6.404959,21.537981,0.074581
2938,25.626435,16.465168,0.017755,5.500000,18.262691,4.250000,394.925932,4.609752,1.250000,2.250000,...,23.750000,0.000000,0.000009,1.000000,2.250000,0.216931,0.002532,8.500000,35.067964,0.136254
539,18.452094,8.313857,0.018688,23.456790,31.669745,2.777778,499.030309,4.980638,27.777778,1.543210,...,8.950617,0.617284,0.000011,14.197531,1.234568,0.200777,0.002004,20.370370,9.734794,0.118544
2329,24.630590,55.528737,0.014984,23.765432,26.517308,0.925926,338.385216,4.482667,23.456790,1.543210,...,16.358025,1.851852,0.000009,16.666667,1.234568,0.223081,0.002955,4.320988,36.032974,0.152439


In [181]:
y_train.head()

1874    False
2800     True
2938     True
539      True
2329    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [182]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

In [183]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [184]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [185]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[76 25]
 [48 49]]
               precision    recall  f1-score   support

 class "real"       0.61      0.75      0.68       101
class "fakes"       0.66      0.51      0.57        97

     accuracy                           0.63       198
    macro avg       0.64      0.63      0.62       198
 weighted avg       0.64      0.63      0.63       198



### SVC (f1=0.64) [to be modified]

In [186]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[70 31]
 [29 68]]
               precision    recall  f1-score   support

 class "real"       0.71      0.69      0.70       101
class "fakes"       0.69      0.70      0.69        97

     accuracy                           0.70       198
    macro avg       0.70      0.70      0.70       198
 weighted avg       0.70      0.70      0.70       198



### LogisticRegression (f1=0.61) [to be modified]

In [187]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[65 36]
 [25 72]]
               precision    recall  f1-score   support

 class "real"       0.72      0.64      0.68       101
class "fakes"       0.67      0.74      0.70        97

     accuracy                           0.69       198
    macro avg       0.69      0.69      0.69       198
 weighted avg       0.70      0.69      0.69       198



### NN - MLPClassifier (f1=0.66) [to be modified]

In [ ]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(30,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

### RandomForestClassifier (good, f1=0.64) [to be modified] 

In [ ]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
) 

### DecisionTreeClassifier (f1=0.59) [to be modified]

In [ ]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test
)

### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [ ]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

### KNeighborsClassifier (f1=0.69) [to be modified]

In [ ]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=3, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)